In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

We've talked about Random Forests. Now it's time to build one.

Here we'll use data from Lending Club to predict the state of a loan given some information about it. You can find the dataset [here](https://www.lendingclub.com/info/download-data.action). We'll use 2015 data. ([Thinkful mirror](https://www.dropbox.com/s/m7z42lubaiory33/LoanStats3d.csv?dl=0))

In [4]:
# Replace the path with the correct path for your data.
y2015 = pd.read_csv(
    'LoanStats3d.csv',
    skipinitialspace=True,
    header=1,nrows=15000
)
#y2015= y2015.sample(frac=0.01)
# Note the warning about dtypes.

c:\users\sakok\appdata\local\programs\python\python36-32\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [48]:
y2015.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,35000,35000,35000,60 months,12.88%,794.21,C,C2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,23850,23850,23850,60 months,17.27%,596.21,D,D3,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,15000,15000,15000,60 months,10.78%,324.50,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,7200,7200,7200,36 months,17.27%,257.67,D,D3,...,NaN,NaN,Cash,Y,May-2018,BROKEN,Nov-2017,2761.0,50.01,10.0
4,NaN,NaN,17475,17475,17475,36 months,13.99%,597.17,C,C4,...,NaN,NaN,Cash,Y,Jan-2018,ACTIVE,Jan-2018,4382.0,44.99,12.0


## The Blind Approach

Now, as we've seen before, creating a model is the easy part. Let's try just using everything we've got and throwing it without much thought into a Random Forest. SKLearn requires the independent variables to be be numeric, and all we want is dummy variables so let's use `get_dummies` from Pandas to generate a dummy variable for every categorical colummn and see what happens off of this kind of naive approach.

In [23]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)

cross_val_score(rfc, X, Y, cv=5)

MemoryError: 

Did your kernel die? My kernel died.

Guess it isn't always going to be that easy...

Can you think of what went wrong?

(You're going to have to reset your kernel and reload the column, BUT DON'T RUN THE MODEL AGAIN OR YOU'LL CRASH THE KERNEL AGAIN!)

## Data Cleaning

Well, `get_dummies` can be a very memory intensive thing, particularly if data are typed poorly. We got a warning about that earlier. Mixed data types get converted to objects, and that could create huge problems. Our dataset is about 400,000 rows. If there's a bad type there its going to see 400,000 distinct values and try to create dummies for all of them. That's bad. Lets look at all our categorical variables and see how many distinct counts there are...

In [5]:
categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

term
2
int_rate
64
grade
7
sub_grade
35
emp_title
7886
emp_length
11
home_ownership
3
verification_status
3
issue_d
1
loan_status
6
pymnt_plan
2
desc
1
purpose
12
title
12
zip_code
814
addr_state
49
earliest_cr_line
534
revol_util
1037
initial_list_status
2
last_pymnt_d
36
next_pymnt_d
2
last_credit_pull_d
36
application_type
2
verification_status_joint
1
hardship_flag
2
hardship_type
1
hardship_reason
9
hardship_status
3
hardship_start_date
19
hardship_end_date
19
payment_plan_start_date
17
hardship_loan_status
4
disbursement_method
1
debt_settlement_flag
2
debt_settlement_flag_date
24
settlement_status
3
settlement_date
27


Well that right there is what's called a problem. Some of these have over a hundred thousand distinct types. Lets drop the ones with over 30 unique values, converting to numeric where it makes sense. In doing this there's a lot of code that gets written to just see if the numeric conversion makes sense. It's a manual process that we'll abstract away and just include the conversion.

You could extract numeric features from the dates, but here we'll just drop them. There's a lot of data, it shouldn't be a huge problem.

In [6]:
# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

Wonder what was causing the dtype error on the id column, which _should_ have all been integers? Let's look at the end of the file.

In [51]:
y2015.tail()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
14995,NaN,NaN,30000,30000,30000,60 months,15.41,720.18,D,< 1 year,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
14996,NaN,NaN,5000,5000,5000,36 months,6.89,154.14,A,10+ years,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
14997,NaN,NaN,9050,9050,9050,36 months,17.86,326.55,D,5 years,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
14998,NaN,NaN,6400,6400,6400,36 months,7.89,200.23,A,10+ years,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
14999,NaN,NaN,35000,35000,35000,36 months,9.76,1125.42,B,1 year,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Remove two summary rows at the end that don't actually contain data.
y2015 = y2015[:-2]

Now this should be better. Let's try again.

In [8]:
pd.get_dummies(y2015.drop('loan_status', 1))

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,settlement_date_Mar-2018,settlement_date_May-2017,settlement_date_May-2018,settlement_date_Nov-2016,settlement_date_Nov-2017,settlement_date_Nov-2018,settlement_date_Oct-2017,settlement_date_Oct-2018,settlement_date_Sep-2017,settlement_date_Sep-2018
0,NaN,NaN,35000,35000,35000,12.88,794.21,106000.0,17.36,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,NaN,23850,23850,23850,17.27,596.21,68046.0,24.71,1,...,0,0,0,0,0,0,0,0,0,0
2,NaN,NaN,15000,15000,15000,10.78,324.50,65000.0,29.28,0,...,0,0,0,0,0,0,0,0,0,0
3,NaN,NaN,7200,7200,7200,17.27,257.67,20000.0,13.21,1,...,0,0,0,0,1,0,0,0,0,0
4,NaN,NaN,17475,17475,17475,13.99,597.17,39750.0,23.58,0,...,0,0,0,0,0,0,0,0,0,0
5,NaN,NaN,20000,20000,20000,5.32,602.30,65000.0,18.83,0,...,0,0,0,0,0,0,0,0,0,0
6,NaN,NaN,18000,18000,18000,7.49,559.83,75000.0,25.65,0,...,0,0,0,0,0,0,0,0,0,0
7,NaN,NaN,16000,16000,16000,12.88,363.07,90000.0,6.39,0,...,0,0,0,0,0,0,0,0,0,0
8,NaN,NaN,24000,24000,24000,9.80,507.58,77900.0,10.24,0,...,0,0,0,0,0,0,0,0,0,0
9,NaN,NaN,10000,10000,10000,12.88,336.37,91392.0,9.44,0,...,0,0,0,0,0,0,0,0,0,0


It finally works! We had to sacrifice sub grade, state address and description, but that's fine. If you want to include them you could run the dummies independently and then append them back to the dataframe.

## Second Attempt

Now let's try this model again.

We're also going to drop NA columns, rather than impute, because our data is rich enough that we can probably get away with it.

This model may take a few minutes to run.

In [10]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)
X

#cross_val_score(rfc, X, Y, cv=10)

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,...,settlement_date_Mar-2018,settlement_date_May-2017,settlement_date_May-2018,settlement_date_Nov-2016,settlement_date_Nov-2017,settlement_date_Nov-2018,settlement_date_Oct-2017,settlement_date_Oct-2018,settlement_date_Sep-2017,settlement_date_Sep-2018
0,35000,35000,35000,12.88,794.21,106000.0,17.36,0,0,8,...,0,0,0,0,0,0,0,0,0,0
1,23850,23850,23850,17.27,596.21,68046.0,24.71,1,0,12,...,0,0,0,0,0,0,0,0,0,0
2,15000,15000,15000,10.78,324.50,65000.0,29.28,0,0,18,...,0,0,0,0,0,0,0,0,0,0
3,7200,7200,7200,17.27,257.67,20000.0,13.21,1,1,8,...,0,0,0,0,1,0,0,0,0,0
4,17475,17475,17475,13.99,597.17,39750.0,23.58,0,0,7,...,0,0,0,0,0,0,0,0,0,0
5,20000,20000,20000,5.32,602.30,65000.0,18.83,0,1,12,...,0,0,0,0,0,0,0,0,0,0
6,18000,18000,18000,7.49,559.83,75000.0,25.65,0,1,12,...,0,0,0,0,0,0,0,0,0,0
7,16000,16000,16000,12.88,363.07,90000.0,6.39,0,0,8,...,0,0,0,0,0,0,0,0,0,0
8,24000,24000,24000,9.80,507.58,77900.0,10.24,0,1,9,...,0,0,0,0,0,0,0,0,0,0
9,10000,10000,10000,12.88,336.37,91392.0,9.44,0,0,13,...,0,0,0,0,0,0,0,0,0,0


In [55]:
y2015['loan_status'].unique()

array(['Charged Off', 'Current', 'Fully Paid', 'Late (31-120 days)',
       'Late (16-30 days)', 'In Grace Period'], dtype=object)

The score cross validation reports is the accuracy of the tree. Here we're about 98% accurate.

That works pretty well, but there are a few potential problems. Firstly, we didn't really do much in the way of feature selection or model refinement. As such there are a lot of features in there that we don't really need. Some of them are actually quite impressively useless.

There's also some variance in the scores. The fact that one gave us only 93% accuracy while others gave higher than 98 is concerning. This variance could be corrected by increasing the number of estimators. That will make it take even longer to run, however, and it is already quite slow.

## DRILL: Third Attempt

So here's your task. Get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

You'll want to do a few things in this process. First, dive into the data that we have and see which features are most important. This can be the raw features or the generated dummies. You may want to use PCA or correlation matrices.

Can you do it without using anything related to payment amount or outstanding principal? How do you know?

In [16]:
# Your code here.
#let's take a look first
#df = y2015.iloc[:, :111]
#corr = df.corr()


from sklearn.decomposition import PCA
pca = PCA(n_components=15)

X_pca = pca.fit_transform(X)
df2 = pd.DataFrame({"1": X_pca[:,0],"2": X_pca[:,1],"3": X_pca[:,2],"4": X_pca[:,3],
                   "5": X_pca[:,4],"6": X_pca[:,5],"7": X_pca[:,6],"8": X_pca[:,7],\
                   "9": X_pca[:,8],"10": X_pca[:,9],"11": X_pca[:,10],"12": X_pca[:,11],
                   "13": X_pca[:,12],"14": X_pca[:,13],"15": X_pca[:,14]})
#new_corr = df2.corr()
#sns.heatmap(new_corr)




In [12]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = df2
Y = y2015['loan_status']


cross_val_score(rfc, X, Y, cv=10)

c:\users\sakok\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\sakok\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\sakok\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
c:\users\sakok\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "

array([ 0.88423154,  0.87292083,  0.88548602,  0.86750999,  0.88674217,
        0.88058706,  0.87641951,  0.88777555,  0.88109552,  0.87708751])